In [41]:
import random

import cv2
import numpy as np

Download images from directory.

In [42]:
from image import read_image, show_image

In [43]:
images, image_names = read_image("InputImages")

In [44]:
image_names

['1633601732_6-p-foto-doma-snizu-vverkh-foto-6.jpg',
 '1633601797_20-p-foto-doma-snizu-vverkh-foto-33.jpg',
 '1633601733_15-p-foto-doma-snizu-vverkh-foto-20.jpg',
 '1633601728_17-p-foto-doma-snizu-vverkh-foto-25.jpg',
 '1633601806_42-p-foto-doma-snizu-vverkh-foto-68.jpg',
 '1633601775_37-p-foto-doma-snizu-vverkh-foto-59.jpg',
 '1633601772_12-p-foto-doma-snizu-vverkh-foto-15.jpg',
 '1633601806_44-p-foto-doma-snizu-vverkh-foto-70.jpg',
 'horizon-cloud-sky-sunlight-sqpc.jpeg',
 '1633601784_50-p-foto-doma-snizu-vverkh-foto-78.jpg',
 '1633601774_48-p-foto-doma-snizu-vverkh-foto-75.jpg']

In [45]:
image = images[0]

In [46]:
show_image("OutputImage", image)

# Stratification

Get lines from the image.

In [22]:
from line import get_lines, draw_line

In [23]:
lines = get_lines(image)
len(lines)

8

Plot lines.

In [24]:
ouput_image = image.copy()
# Getting the lines form the image
for line in lines:
    r = random.randint(0, 256)
    g = random.randint(0, 256)
    b = random.randint(0, 256)
    draw_line(ouput_image, line, (r, g, b))
show_image("OutputImage", ouput_image)

Cluster lines to find vanishing points.

In [25]:
from clustering import cluster_lines

In [26]:
n_clusters = 3

In [27]:
clusters = cluster_lines(lines, n_clusters=n_clusters)
clusters

defaultdict(list,
            {2: [array([[-20.      ,   2.024582]], dtype=float32),
              array([[-16.      ,   2.024582]], dtype=float32),
              array([[123.       ,   2.0943952]], dtype=float32)],
             1: [array([[371.       ,   1.0821041]], dtype=float32),
              array([[368.       ,   1.0821041]], dtype=float32),
              array([[289.       ,   1.1170107]], dtype=float32)],
             0: [array([[701.       ,   0.9250245]], dtype=float32),
              array([[711.       ,   0.9075712]], dtype=float32)]})

Show image with clustered lines and find vanishing points.

In [29]:
from homography import get_vanishing_point, get_distant_points, GetVanishingPoint

In [30]:
vanishing_points = [get_vanishing_point(clusters[c]) for c in clusters]
vanishing_points

[array([-132142.6 ,  244221.34], dtype=float32),
 array([51955.97 , 96889.234], dtype=float32),
 array([ 454.5628 , -348.79706], dtype=float32)]

In [31]:
vanishing_points = [GetVanishingPoint(clusters[c]) for c in clusters]
vanishing_points

[[-0.9825816779958353, -2.6568469860760273],
 [0.9922327599404025, -4.718030352214626],
 [-2.118462210710019, 2.4385799958539014]]

In [33]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(n_clusters)]

output_image = image.copy()
for c in clusters:
    vp = vanishing_points[c]
    print(vp, colors[c])
    for i, line in enumerate(clusters[c]):
        draw_line(output_image, line, colors[c])
    cv2.circle(output_image, (int(vp[0]), int(vp[1])), 10, colors[c], -1)

show_image("OutputImage", output_image)  

[-2.118462210710019, 2.4385799958539014] (160, 124, 217)
[0.9922327599404025, -4.718030352214626] (209, 165, 194)
[-0.9825816779958353, -2.6568469860760273] (1, 152, 249)


In [34]:
two_points, _ = get_distant_points(vanishing_points)
two_points

[[0.9922327599404025, -4.718030352214626],
 [-2.118462210710019, 2.4385799958539014]]

Perform stratification.

In [35]:
vanishing_line = np.cross([*two_points[0], 1], [*two_points[1], 1])
vanishing_line

array([-7.15661035, -3.11069497, -7.57533005])

In [36]:
H1 = np.eye(3)
H1[2] = vanishing_line
H1[2] /= np.linalg.norm(H1, axis=1)[2]
#H1[2] /= H1[2][2]
print(H1)

[[ 1.          0.          0.        ]
 [ 0.          1.          0.        ]
 [-0.65804157 -0.2860246  -0.69654233]]


In [37]:
new_image = cv2.warpPerspective(image, H1, (image.shape[1] * 10, image.shape[0] * 10))

In [40]:
show_image("OutputImage", new_image)

# Metric rectification

Get lines from transformed images.

In [ ]:
new_image = image.copy()

In [ ]:
new_lines = get_lines(new_image)
len(new_lines)

In [ ]:
output_image = new_image.copy()
for line in new_lines:
    r = random.randint(0, 256)
    g = random.randint(0, 256)
    b = random.randint(0, 256)
    draw_line(output_image, line, (r, g, b))
show_image("OutputImage", output_image)

In [ ]:
new_clusters = cluster_lines(new_lines, n_clusters=n_clusters)
new_clusters

In [ ]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(n_clusters)]

output_image = new_image.copy()
for c in new_clusters:
    for line in new_clusters[c]:
        draw_line(output_image, line, colors[c])

show_image("OutputImage", output_image)  

Find orthogonal lines.

In [ ]:
from line import find_orthogonal_lines

In [ ]:
orthogonal_lines = find_orthogonal_lines(new_clusters)
orthogonal_lines

In [ ]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(len(orthogonal_lines))]

output_image = new_image.copy()
for i, lines in enumerate(orthogonal_lines):
    for line in lines:
        draw_line(output_image, line, colors[i])

show_image("OutputImage", output_image)

Make linear equation from 3 orthogonal lines.

In [ ]:
from line import get_homogeneous_coordinates

In [ ]:
s_true = np.array([1, 1, 0])
best_u, best_s = [], []
best_dist = float('inf')

In [ ]:
from scipy.linalg import null_space

In [ ]:
for i in range(len(orthogonal_lines)):
    for j in range(i+1, len(orthogonal_lines)):
        A = []
        pair = orthogonal_lines[i]
        l, m = get_homogeneous_coordinates(pair[0]), get_homogeneous_coordinates(pair[1])
        A.append([l[0] * m[0], l[1] * m[1], l[0] * m[1] + l[1] * m[0]])
        
        pair = orthogonal_lines[j]
        l, m = get_homogeneous_coordinates(pair[0]), get_homogeneous_coordinates(pair[1])
        A.append([l[0] * m[0], l[1] * m[1], l[0] * m[1] + l[1] * m[0]])
        
        s = null_space(A)
        
        C_dual = np.zeros((3, 3))
        C_dual[0][0], C_dual[0][1] = s[0], s[1]
        C_dual[1][0], C_dual[1][1] = s[1], s[2]
        C_dual[2][2] = 1
        
        u, s, vh = np.linalg.svd(C_dual)
        dist = np.linalg.norm(s_true - s)
        if dist < best_dist:
            best_dist = dist
            best_u = u
            best_s = s

In [ ]:
best_u, best_s

In [ ]:
np.linalg.norm(best_u)

In [ ]:
H = best_u @ H1

In [ ]:
np.linalg.norm(H, axis=0)

In [ ]:
output_image = cv2.warpPerspective(new_image, best_u, (new_image.shape[1], new_image.shape[0]))

In [51]:
show_image("OutputImage", output_image)

In [ ]:
output_image = cv2.warpPerspective(image, H, (new_image.shape[1], new_image.shape[0]))

In [ ]:
show_image("OutputImage", output_image)